<a href="https://colab.research.google.com/github/akshi-saxena/IR-Project/blob/main/IRProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing libraries

In [152]:
import pandas as pd
import numpy as np

import nltk
from nltk import corpus
nltk.download('stopwords')

# from nltk.stem.porter import PorterStemmer

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Acquiring Data

In [100]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [101]:
# link_train = 'https://drive.google.com/file/d/1-52Y3wUADsknJDxJ3y_ejZ9ULrZHYcmM'
link_train = 'https://drive.google.com/file/d/1JnzRS-DX5zwxWpzIKf_RpM3fblt-yzcC'

In [102]:
data, id = link_train.split('/d/')
print (id) # Verify that you have everything after '/d/'

1JnzRS-DX5zwxWpzIKf_RpM3fblt-yzcC


Create train dataframe

In [103]:
train = drive.CreateFile({'id':id}) 
train.GetContentFile('training.csv')  
train_df = pd.read_csv('training.csv')

In [104]:
train_df.head()

,Id,title,text,subject,date,label
0,13970,WATCH: Paul Manafort Gets DESTROYED On CNN Fo...,"Paul Manafort, the chair of Donald Trump s cam...",News,"August 14, 2016",1
1,41668,REFUGEE BUSINESS IS CASH COW FOR LUTHERAN CHAR...,How very charitable of the Lutheran and Cath...,left-news,"May 8, 2015",1
2,26810,Trump’s Excuse For All His Sexist Statements ...,In case you ve been living under a rock for yo...,News,"March 28, 2016",1
3,30967,'Jihadi Gran' gets 10 years after joining son ...,PARIS (Reuters) - A court on Friday sentenced ...,worldnews,"October 6, 2017",0
4,26072,New Czech government to seek confidence vote o...,PRAGUE (Reuters) - The Czech government will s...,worldnews,"December 13, 2017",0


In [105]:
# Records in training data
len(train_df)

31428

Create test dataframe

In [106]:
# link_test = 'https://drive.google.com/file/d/1NjOcOCKVMlFyJXnPqIxBv3IWZAKMnwM0'
link_test = 'https://drive.google.com/file/d/1kO4FkbqMdpa1_fDVWHqvQ7CxfxS5AMwE'

In [107]:
data, id = link_test.split('/d/')
print (id) # Verify that you have everything after '/d/'

1kO4FkbqMdpa1_fDVWHqvQ7CxfxS5AMwE


In [108]:
train = drive.CreateFile({'id':id}) 
train.GetContentFile('test.csv')  
test_df = pd.read_csv('test.csv')

In [109]:
test_df.head()

,Id,title,text,subject,date
0,22216,Five killed in sectarian attack in Pakistan,"QUETTA, Pakistan (Reuters) - (This October 9 s...",worldnews,"October 9, 2017"
1,27917,White House narrows search to three for Suprem...,"WASHINGTON/AUSTIN, Texas (Reuters) - The White...",politicsNews,"March 11, 2016"
2,25007,THE DEATH OF P.C. POLICE? Trump’s DOJ Makes Ma...,The Justice Department on Thursday dropped the...,politics,"Jun 30, 2017"
3,1377,"House, Senate Republicans face challenge over ...",WASHINGTON (Reuters) - U.S. Republicans in Con...,politicsNews,"December 5, 2017"
4,32476,China complains about Taiwan content in U.S. d...,BEIJING (Reuters) - China said on Thursday it ...,worldnews,"December 14, 2017"


In [110]:
#Records in test dataframe
len(test_df)

13470

### Dropping null values

In [111]:
train_df = train_df.dropna()
train_df

,Id,title,text,subject,date,label
0,13970,WATCH: Paul Manafort Gets DESTROYED On CNN Fo...,"Paul Manafort, the chair of Donald Trump s cam...",News,"August 14, 2016",1
1,41668,REFUGEE BUSINESS IS CASH COW FOR LUTHERAN CHAR...,How very charitable of the Lutheran and Cath...,left-news,"May 8, 2015",1
2,26810,Trump’s Excuse For All His Sexist Statements ...,In case you ve been living under a rock for yo...,News,"March 28, 2016",1
3,30967,'Jihadi Gran' gets 10 years after joining son ...,PARIS (Reuters) - A court on Friday sentenced ...,worldnews,"October 6, 2017",0
4,26072,New Czech government to seek confidence vote o...,PRAGUE (Reuters) - The Czech government will s...,worldnews,"December 13, 2017",0
...,...,...,...,...,...,...
31423,11284,CHICAGO TRUMP RALLY CANCELLED: Radicals And BL...,"A huge rally of 10,000 Chicagoans didn t get t...",politics,"Mar 11, 2016",1
31424,44732,Trump supports completion of Dakota Access Pip...,WASHINGTON (Reuters) - U.S. President-elect Do...,politicsNews,"December 5, 2016",0
31425,38158,Obama Can’t Stop Winning As New Jobs Report S...,"Today, I would hate to be a Republican strateg...",News,"January 8, 2016",1
31426,860,Turkey bank regulator dismisses 'rumors' after...,ANKARA (Reuters) - Turkey s banking regulator ...,worldnews,"October 21, 2017",0


### Removing Stopwords

In [112]:
stop_words = corpus.stopwords.words('english')
def remove_stopwords(text):
    return ' '.join(word.lower() for word in text.split(' ') if word not in stop_words)

train_df.text = train_df.text.apply(remove_stopwords)

In [113]:
train_df.head()

,Id,title,text,subject,date,label
0,13970,WATCH: Paul Manafort Gets DESTROYED On CNN Fo...,"Paul Manafort, chair Donald Trump campaign, go...",News,"August 14, 2016",1
1,41668,REFUGEE BUSINESS IS CASH COW FOR LUTHERAN CHAR...,How charitable Lutheran Catholic churches br...,left-news,"May 8, 2015",1
2,26810,Trump’s Excuse For All His Sexist Statements ...,"In case living rock entire life, especially ca...",News,"March 28, 2016",1
3,30967,'Jihadi Gran' gets 10 years after joining son ...,PARIS (Reuters) - A court Friday sentenced rad...,worldnews,"October 6, 2017",0
4,26072,New Czech government to seek confidence vote o...,PRAGUE (Reuters) - The Czech government seek v...,worldnews,"December 13, 2017",0


### Removing junk characters

In [114]:
train_df.text = train_df.text.str.replace('\d+','')

### Lemmatization

In [115]:
# ps = PorterStemmer()
# def stemming(text):
#     return ' '.join(ps.stem(word) for word in text.split(' ') if word not in stop_words)

# train_df.text = train_df.text.apply(stemming)

In [116]:
lemmatizer = WordNetLemmatizer()
def lemmatizing(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split(' ') )

train_df.text = train_df.text.apply(lemmatizing)

### Calculate TF-IDF

In [117]:
tfidf =TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = tfidf.fit_transform(train_df.text).toarray()

In [118]:
X.shape

(31428, 5000)

In [119]:
y = train_df.label

In [120]:
y[:10]

0    1
1    1
2    1
3    0
4    0
5    1
6    1
7    1
8    0
9    0
Name: label, dtype: int64

Create train-validation data split

In [121]:
x_train, x_dev, y_train, y_dev = train_test_split(X,y,train_size=0.25, random_state = 12)

### Naive Bayes Classifier

In [122]:
classifier = MultinomialNB().fit(x_train,y_train)
preds = classifier.predict(x_dev)

In [123]:
preds[:10]

array([0, 1, 1, 1, 1, 0, 0, 0, 0, 1])

### Calculate accuracy

In [124]:
def accuracy(preds, actual):
  acc = 0 
  for i,j in zip(preds, actual):
    if i == j:
      acc += 1

  return acc/len(actual)

accuracy(preds, y_dev)

0.9492596835093972

### Calculate Precision and Recall

In [125]:
def precision(gold_labels, predicted_labels, positive_label = '1'):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(gold_labels, predicted_labels):
      if i == positive_label:
          if j == i:
              tp += 1
          else:
              fn += 1
      else:
          if j == i:
              tn += 1
          else:
              fp += 1
    return (tp) / (tp + fp)


def recall(gold_labels, predicted_labels, positive_label = '1'):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(gold_labels, predicted_labels):
        if i == positive_label:
            if i == j:
                tp += 1
            else:
                fn += 1
        else:
            if i == j:
                tn += 1
            else:
                fp += 1
    return (tp) / (tp + fn)
    

In [126]:
precision(y_dev.tolist(), list(preds.flatten()), 1)

0.9544863459037711

In [127]:
recall(y_dev.tolist(), list(preds.flatten()), 1)

0.948626817447496

### Getting Topical relevance

In [139]:
# Topics in the dataset
train_df["subject"].value_counts()

politicsNews       7863
worldnews          7088
News               6378
politics           4832
left-news          3096
Government News    1089
US_News             546
Middle-east         536
Name: subject, dtype: int64

In [140]:
#Convert topics to numeric values
train_df["subject"] = train_df["subject"].astype('category')
train_df["subject_label"] = train_df["subject"].cat.codes

In [141]:
train_df

,Id,title,text,subject,date,label,subject_label
0,13970,WATCH: Paul Manafort Gets DESTROYED On CNN Fo...,"Paul Manafort, chair Donald Trump campaign, go...",News,"August 14, 2016",1,2
1,41668,REFUGEE BUSINESS IS CASH COW FOR LUTHERAN CHAR...,How charitable Lutheran Catholic church brin...,left-news,"May 8, 2015",1,4
2,26810,Trump’s Excuse For All His Sexist Statements ...,"In case living rock entire life, especially ca...",News,"March 28, 2016",1,2
3,30967,'Jihadi Gran' gets 10 years after joining son ...,PARIS (Reuters) - A court Friday sentenced rad...,worldnews,"October 6, 2017",0,7
4,26072,New Czech government to seek confidence vote o...,PRAGUE (Reuters) - The Czech government seek v...,worldnews,"December 13, 2017",0,7
...,...,...,...,...,...,...,...
31423,11284,CHICAGO TRUMP RALLY CANCELLED: Radicals And BL...,"A huge rally , Chicagoans get hear candidate s...",politics,"Mar 11, 2016",1,5
31424,44732,Trump supports completion of Dakota Access Pip...,WASHINGTON (Reuters) - U.S. President-elect Do...,politicsNews,"December 5, 2016",0,6
31425,38158,Obama Can’t Stop Winning As New Jobs Report S...,"Today, I would hate Republican strategist. Why...",News,"January 8, 2016",1,2
31426,860,Turkey bank regulator dismisses 'rumors' after...,ANKARA (Reuters) - Turkey banking regulator ur...,worldnews,"October 21, 2017",0,7


Calculate TF-IDF and pass to classifier

In [142]:
X_multi = tfidf.fit_transform(train_df.text).toarray()
y_multi = train_df.subject_label

In [179]:
x_multi_train,x_multi_dev,y_multi_train,y_multi_dev = train_test_split(X_multi,y_multi,train_size=0.25, random_state = 32)
multi_classifier = MultinomialNB().fit(x_multi_train,y_multi_train)
preds_multi = multi_classifier.predict(x_multi_dev)

In [180]:
accuracy(preds_multi, y_multi_dev)

0.7042552288829493

### Test Data prediction

In [ ]:
test_df = test_df.dropna()
test_df.text = test_df.text.apply(remove_stopwords)
test_df.text = test_df.text.apply(lemmatizing)
test_df.text

0        quetta, pakistan (reuters) - (thi octob 9 stor...
1        washington/austin, texa (reuters) - the white ...
2        the justic depart thursday drop legal challeng...
3        washington (reuters) - u.s. republican congres...
4        beij (reuters) - china said thursday complain ...
                               ...                        
13465    ron reagan, late presid reagan son, understand...
13466    A prolif trump foe found viral fame boast aban...
13467    washington (reuters) - u.s. presid donald trum...
13468    beij (reuters) - A young chines climb enthusia...
13469    hillari lost lead swing state oh, nv, IA FL si...
Name: text, Length: 13470, dtype: object

In [ ]:
X_test = tfidf.fit_transform(train_df.text).toarray()

In [ ]:
X_test.shape

(31428, 5000)

In [ ]:
pred_test = classifier.predict(X_test)
pred_test[:100]

array([1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1])

In [ ]:
multi_classifier.predict(X_test)

### Calculate query - doc similarity

In [145]:
titles = list(train_df.title)
docs = list(train_df.text)
ids = list(train_df.Id)

In [233]:
X = tfidf.fit_transform(docs)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=tfidf.get_feature_names_out())

### Calculate similarity score of query and document

In [234]:
def get_similar_articles(q, df, n):
  print("query:", q)
  print("search results: ")
  ans_df = pd.DataFrame()
  ans_title = []
  ans_sim = []
  ans_id = []

  # Convert the query become a vector
  q = [q]
  q_vec = tfidf.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  
  # Calculate the similarity
  for i in range(df.shape[1]):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      # print("Cosine Similarity:", v)
      ans_sim.append(v)
      # print(titles[k])
      ans_title.append(titles[k])
      ans_id.append(ids[k])
      # print()
  ans_df['id'] = ans_id
  ans_df['title'] = ans_title
  ans_df['sim'] = ans_sim
  return ans_df[:n]

### Basic Search Engine

In [245]:
# Add The Query
q1 = 'trump'

# Get 100 similar documents
ans = get_similar_articles(q1, df, 100)

query: trump
search results: 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [246]:
ans

,id,title,sim
0,20109,BUSTED: Steel Unions FURIOUS At Trump For Des...,0.342352
1,16178,Greedy Trump Cuts Takes Housing Money Away Fr...,0.318355
2,39856,Trump Just Got DESTROYED By J.K. Rowling For ...,0.318006
3,23254,Gawker Fools Trump Into Quoting Genocidal Man...,0.281729
4,34361,Trump hugs Harvey's victims in test of preside...,0.272363
...,...,...,...
95,30162,"White House aims to speed U.S. drone, wireless...",0.033781
96,26891,LOL! HYPOCRITE HILLARY Gives Speech On Evils O...,0.033073
97,11709,CONTROVERSY AT SCHOOL After Brave Student Wear...,0.030954
98,7387,Sign-up pace slows in third week of 2018 Obama...,0.030706


In [240]:
# Get topic of query
def get_query_class(q):
  q = [q]
  q_vec = tfidf.transform(q).toarray()
  subj_class = multi_classifier.predict(q_vec)
  print(subj_class)
  return subj_class

In [241]:
query_topic = get_query_class(q1)

# Get similar documents from predicted label dataset
result_ids = list(ans.id)
ans_labels = train_df[[True if i in result_ids else False for i in train_df.Id] ][["Id","subject_label","label"]]
ans_labels.head()

[2]


,Id,subject_label,label
0,13970,2,1
2,26810,2,1
5,7209,5,1
7,41384,5,1
8,34361,6,0


In [242]:
docs_retreived = pd.merge(ans,ans_labels, left_on=["id"], right_on=["Id"])
docs_retreived

,id,title,sim,Id,subject_label,label
0,20109,BUSTED: Steel Unions FURIOUS At Trump For Des...,0.342352,20109,2,1
1,16178,Greedy Trump Cuts Takes Housing Money Away Fr...,0.318355,16178,2,1
2,39856,Trump Just Got DESTROYED By J.K. Rowling For ...,0.318006,39856,2,1
3,23254,Gawker Fools Trump Into Quoting Genocidal Man...,0.281729,23254,2,1
4,34361,Trump hugs Harvey's victims in test of preside...,0.272363,34361,6,0
...,...,...,...,...,...,...
95,30162,"White House aims to speed U.S. drone, wireless...",0.033781,30162,6,0
96,26891,LOL! HYPOCRITE HILLARY Gives Speech On Evils O...,0.033073,26891,5,1
97,11709,CONTROVERSY AT SCHOOL After Brave Student Wear...,0.030954,11709,5,1
98,7387,Sign-up pace slows in third week of 2018 Obama...,0.030706,7387,6,0


Filter on basis of reliability and topical relevance

In [243]:
class_labs = list(docs_retreived.label)
fact_filtered = docs_retreived[[True if i == 1 else False for i in class_labs]]

subj_labs = list(docs_retreived.subject_label)
topic_filtered = docs_retreived[[True if i == query_topic else False for i in subj_labs]][["id","title"]]

topic_filtered

,id,title
0,20109,BUSTED: Steel Unions FURIOUS At Trump For Des...
1,16178,Greedy Trump Cuts Takes Housing Money Away Fr...
2,39856,Trump Just Got DESTROYED By J.K. Rowling For ...
3,23254,Gawker Fools Trump Into Quoting Genocidal Man...
6,13970,WATCH: Paul Manafort Gets DESTROYED On CNN Fo...
7,26810,Trump’s Excuse For All His Sexist Statements ...
8,5086,Billionaire Richard Branson Breaks Silence Ab...
9,8318,Warren Buffett Knows Exactly Why Trump’s Camp...
10,2932,Hillary’s Campaign Account Hasn’t Tweeted Sin...
11,32545,"CNN And MSNBC Destroy Trump, Black Out His Fa..."
